## Comparing product prices from different sites using web scraping and Sqlite3

<b>Import<b>

In [31]:
import requests
from bs4 import BeautifulSoup
import datetime
import sqlite3
import pandas as pd

<b>Make / Connect to database</b>

In [32]:
conn = sqlite3.connect('prizes')
c = conn.cursor()


<b>Create table</b>

In [33]:
c.execute('''DROP TABLE comparison''')
c.execute('''CREATE TABLE comparison (date DATE, store TEXT, title TEXT, price REAL, stock TEXT)''')

In [34]:
<b>Code to scrape the data</b>

SyntaxError: invalid syntax (1815587590.py, line 1)

In [35]:
def getPriceMM(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'}
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.content, 'html.parser')
    current_date = datetime.datetime.now()
    store = 'Media Markt'
    title = soup.find('h1').text
    price = soup.find('div', {'class': 'StyledBox-sc-1vld6r2-0 goTwsP'}).find('span', {'class': 'ScreenreaderTextSpan-sc-11hj9ix-0 kZCfsu'}).text.replace('undefined', '')
    stock = soup.find('div', {'class': 'StyledAvailabilityHeadingWrapper-sc-901vi5-2 iPyFyN'}).find('span', {'class':'BaseTypo-sc-1jga2g7-0 izkVco StyledInfoTypo-sc-1jga2g7-1 ZaejY StyledAvailabilityTypo-sc-901vi5-7 egVdxU'}).text
    c.execute('''INSERT INTO comparison VALUES(?,?,?,?,?)''', (current_date, store, title, price, stock))
    #print(current_date, store, title, price, stock)
    return

def getPriceBOL(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'}
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.content, 'html.parser')
    current_date = datetime.datetime.now()
    store = 'Game Mania'
    title = soup.find('div', {'class':'pdp-header slot slot--pdp-header js_slot-title'}).find('span', {'class':'u-mr--xs'}).text
    price = soup.find('div', {'class': 'price-block__price'}).find('span', {'class': 'promo-price'}).text.replace('\n', ',99').strip('99,99')
    stock = soup.find('div', {'class': 'buy-block__delivery-text'}).find( 'span').text
    c.execute('''INSERT INTO comparison VALUES(?,?,?,?,?)''', (current_date, store, title, price, stock))
    #print(current_date, store, title, price, stock)
    return

def getPricePS(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'}
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.content, 'html.parser')
    current_date = datetime.datetime.now()
    store = 'BCC'
    title = soup.find('h1').text.replace('\n',' ')
    price = soup.find('div', {'class': 'psw-l-anchor psw-l-stack-left psw-fill-x psw-p-y-4 psw-p-x-5'}).text.replace('€', '')
    stock = soup.find('div', {'class': 'psw-fade-out psw-l-line-left'}).find('span', {'class':'psw-fill-x psw-t-truncate-1 psw-l-space-x-2'}).text
    c.execute('''INSERT INTO comparison VALUES(?,?,?,?,?)''', (current_date, store, title, price, stock))
    #print(current_date, store, title, price, stock)
    return


In [36]:
getPriceMM('https://www.mediamarkt.nl/nl/product/_ps5-fifa-23-ps5-1734919.html')
getPriceBOL('https://www.bol.com/nl/nl/p/fifa-23-ps4/9300000116890730/?Referrer=ADVNLGOO002012-G-138517438515-S-1671973218393-9300000116890730&gclid=CjwKCAjwo_KXBhAaEiwA2RZ8hKHems2ldeJTT3_OzP3h6J7RjnEruty1hMDEZSk1Rifa1DAnrGkMxBoCNsYQAvD_BwE')
getPricePS('https://store.playstation.com/nl-nl/concept/10004336')

In [38]:
conn.commit()
print('complete')



df = pd.read_sql_query( "SELECT* FROM comparison", conn)
print(df)
conn.close()


complete
                         date        store  \
0  2022-11-30 17:09:18.305455  Media Markt   
1  2022-11-30 17:09:18.689235   Game Mania   
2  2022-11-30 17:09:18.826966          BCC   

                                      title     price  \
0                   FIFA 23 | PlayStation 5     79.99   
1                             FIFA 23 - PS4  57,99  -   
2  EA SPORTS™ FIFA 23 Standard Edition PS5™     79,99   

                                     stock  
0                      Online op voorraad…  
1  Voor 23:59 uur besteld, vrijdag in huis  
2                   Toevoegen aan wagentje  
